In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
tar: spark-3.4.0-bin-hadoop3.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [1]:
!pip install pyspark findspark

In [2]:
# # Start a SparkSession
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [18]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

In [19]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [20]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
q3_query = """
    SELECT
        YEAR(date) AS sale_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 4
    GROUP BY
        sale_year
"""
q3_result = spark.sql(q3_query)
q3_result.show()

+---------+---------+
|sale_year|avg_price|
+---------+---------+
|     2022|296363.88|
|     2019| 300263.7|
|     2020|298353.78|
|     2021|301819.44|
+---------+---------+



In [21]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
q4_query = """
      SELECT
          YEAR(to_date(CAST(date_built AS STRING), 'yyyy')) AS build_year,
          ROUND(AVG(price), 2) AS avg_price
      FROM
          home_sales
      WHERE
          bedrooms = 3
          AND bathrooms = 3
      GROUP BY
          build_year
    """
q4_result = spark.sql(q4_query)
# Show the result
q4_result.show()

+----------+---------+
|build_year|avg_price|
+----------+---------+
|      2015| 288770.3|
|      2013|295962.27|
|      2014|290852.27|
|      2012|293683.19|
|      2016|290555.07|
|      2010|292859.62|
|      2011|291117.47|
|      2017|292676.79|
+----------+---------+



In [22]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
q5_query = """
    SELECT
        YEAR(to_date(CAST(date_built AS STRING), 'yyyy')) AS build_year,
        ROUND(AVG(price), 2) AS avg_price
    FROM
        home_sales
    WHERE
        bedrooms = 3
        AND bathrooms = 3
        AND floors = 2
        AND sqft_living >= 2000
    GROUP BY
        build_year
"""
q5_result = spark.sql(q5_query)
# Show the result
q5_result.show()


+----------+---------+
|build_year|avg_price|
+----------+---------+
|      2015|297609.97|
|      2013|303676.79|
|      2014|298264.72|
|      2012|307539.97|
|      2016| 293965.1|
|      2010|285010.22|
|      2011|276553.81|
|      2017|280317.58|
+----------+---------+



In [29]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.



start_time = time.time()
q6_query = """
    SELECT
        ROUND(AVG(view), 2) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
"""
q6_result = spark.sql(q6_query)
q6_result.show()
# Measure and print the query runtime
end_time = time.time()
query_runtime = end_time - start_time
print("--- %s seconds ---" % query_runtime)

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.260129451751709 seconds ---


In [30]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [31]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [32]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

q9_query = """
    SELECT
        ROUND(AVG(view), 2) AS avg_view_rating
    FROM
        home_sales
    WHERE
        price >= 350000
"""
q9_result = spark.sql(q9_query)
q9_result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.2304065227508545 seconds ---


In [34]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
output_path = "home_sales_partitioned.parquet"
df.write.partitionBy("date_built").parquet(output_path)

In [38]:
# 11. Read the parquet formatted data.
parquet_path = "home_sales_partitioned.parquet"
parquet_df = spark.read.parquet(parquet_path)

In [39]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("parquet_data")

In [41]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

# SQL Query
parquet_query = """
    SELECT ROUND(AVG(view), 2) AS avg_view_rating
    FROM parquet_data
    WHERE price >= 350000
"""

# Execute the SQL query
parquet_result = spark.sql(parquet_query)
# Show the result
parquet_result.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------+
|avg_view_rating|
+---------------+
|          32.26|
+---------------+

--- 0.5374300479888916 seconds ---


In [42]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [43]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False